<a href="https://colab.research.google.com/github/jodog0412/DACON/blob/main/TRAVEL_BOOL_ANALYSIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab/TRAVEL_PRODUCT_ANALYSIS
!ls

# 1. 데이터 로드

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
train = pd.read_csv('dataset/train.csv')
test = pd.read_csv('dataset/test.csv')
sample_submission = pd.read_csv('dataset/sample_submission.csv')
train.head()
# train.isna().sum()

# 2. 데이터 결측치 처리

In [ ]:
train_na = train.copy()
test_na = test.copy()

# 0 으로 채우는 경우
train_na.DurationOfPitch = train_na.DurationOfPitch.fillna(0)
test_na.DurationOfPitch=test_na.DurationOfPitch.fillna(0)


# mean 값으로 채우는 경우
mean_cols = ['Age',
             'NumberOfFollowups',
             'PreferredPropertyStar',
             'NumberOfTrips',
             'NumberOfChildrenVisiting',
             'MonthlyIncome']
             
for col in mean_cols:
    train_na[col] = train_na[col].fillna(train[col].mean())
    test_na[col] = test_na[col].fillna(test[col].mean())

# "Unknown"으로 채우는 경우
train_na.TypeofContact = train_na.TypeofContact.fillna("Unknown")
test_na.TypeofContact = test_na.TypeofContact.fillna("Unknown")

# 3. 문자열 변수 전처리

In [ ]:
object_columns = train.columns[train.dtypes == 'object']

from sklearn.preprocessing import LabelEncoder
train_enc = train_na.copy()
test_enc=test_na.copy()
for o_col in object_columns:
    encoder = LabelEncoder()
    encoder.fit(train_enc[o_col])
    train_enc[o_col] = encoder.transform(train_enc[o_col])
    test_enc[o_col] = encoder.transform(test_enc[o_col])

train_enc.info()
train_enc.describe(include="number")
# train_enc["MonthlyIncome"].hist(bins=100)

# 4. 스케일링

In [ ]:
from sklearn.preprocessing import MinMaxScaler
train_scale = train_enc.copy()
test_scale=test_enc.copy()
scaler = MinMaxScaler()
scaler.fit(train_scale[['Age',
                        'DurationOfPitch',
                        'MonthlyIncome']])
train_scale[['Age','DurationOfPitch','MonthlyIncome']] = scaler.transform(train_scale[['Age', 
                                                                                       'DurationOfPitch', 
                                                                                       'MonthlyIncome']])
test_scale[['Age', 'DurationOfPitch', 'MonthlyIncome']] = scaler.transform(test_scale[['Age',
                                                                                       'DurationOfPitch',
                                                                                       'MonthlyIncome']])
# 결과를 확인합니다.
train_scale.info()

# 5. 머신러닝

## 1) RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
x_train = train_scale.drop(columns=['id','ProdTaken'])
y_train = train_scale[['ProdTaken']]
x_test = test_scale.drop(columns=['id'])

model = RandomForestClassifier()
model.fit(x_train,y_train)
prediction = model.predict(x_test)

## 2. DEEP LEARNING

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

x_train = train_scale.drop(columns=['id','ProdTaken'])
y_train = train_scale[['ProdTaken']]
x_test = test_scale.drop(columns=['id'])

config = {
    'EPOCHS': 10000, #에포크
    'LEARNING_RATE':2e-2, #학습률
    'BATCH_SIZE':12, #배치사이즈
    'SEED':41, #시드
}
X=torch.tensor(x_train.values)
Y=torch.tensor(y_train.values)
model = nn.Sequential(
    nn.LeakyReLU(),
    nn.Linear(18, 32),
    nn.LeakyReLU(),
    nn.Linear(32, 64),
    nn.LeakyReLU(),
    nn.Linear(64, 1),
)
optimizer = torch.optim.Adam(model.parameters(),lr=5e-3)
loss_fn = F.binary_cross_entropy_with_logits
losses=[]
accur=[]
Y_pred=[]
for epoch in range(config['EPOCHS']):
    optimizer.zero_grad() 
    y_pred = model(X.float())  
    acc = (y_pred.detach().numpy().round() == y_train.values).mean()
    loss = loss_fn(y_pred, Y.float())
    loss.backward()  
    optimizer.step()  
    losses.append(loss.item())
    accur.append(acc.item())
    Y_pred.append(y_pred)
plt.plot(losses)

In [ ]:
model.eval()
Y_pred=model(X.float())>torch.tensor([0.5])
cf = confusion_matrix(y_train, Y_pred.detach().numpy())
print(cf)
acc=(cf[0][0]+cf[1][1])/(cf[0][0]+cf[0][1]+cf[1][0]+cf[1][1])
print(acc)

In [ ]:
sample_submission['ProdTaken'] = prediction
sample_submission.head()
sample_submission.to_csv('submission.csv',index = False)